# Building Agents with Tools in Python - Solution

This notebook demonstrates how to build AI agents that can call external tools using OpenAI's function calling capabilities.

## Learning Objectives
- Understand OpenAI function calling mechanics
- Learn to create tool schemas with proper validation
- Practice forcing specific tool execution with `tool_choice`
- Implement error handling for tool calls
- Work with financial calculation tools

## Tools Implemented
1. **Compound Interest Calculator**: Calculate investment growth over time
2. **Currency Converter**: Convert between USD, EUR, and GBP

In [7]:
# Environment Setup and Imports
import os, sys, json, argparse
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables and initialize OpenAI client for Vocareum
load_dotenv()
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")  # Load from .env file
)

# System prompt for the financial assistant
SYSTEM = "You are a helpful financial assistant. Always produce concise, factual answers."

print("✅ Environment setup complete!")
print(f"✅ Using Vocareum OpenAI endpoint")
print(f"✅ API key loaded: {'YES' if os.getenv('OPENAI_API_KEY') else 'NO'}")

✅ Environment setup complete!
✅ Using Vocareum OpenAI endpoint
✅ API key loaded: YES


In [8]:
# Tool Implementations

def calculate_compound_interest(principal: float, rate: float, time: int, compounds_per_year: int = 12) -> dict:
    """
    Calculate compound interest for financial planning.
    Return dict: {"principal": float, "final_amount": float, "total_interest": float, "rate": float, "time": int}
    """
    # Compound interest formula: A = P(1 + r/n)^(nt)
    final_amount = principal * (1 + rate/100/compounds_per_year) ** (compounds_per_year * time)
    total_interest = final_amount - principal
    
    return {
        "principal": round(principal, 2),
        "final_amount": round(final_amount, 2),
        "total_interest": round(total_interest, 2),
        "rate": rate,
        "time": time,
        "compounds_per_year": compounds_per_year
    }

def convert_currency(amount: float, from_code: str, to_code: str) -> dict:
    """
    Convert amount using a small static table for USD/EUR/GBP.
    Return dict: {"amount": float, "rate": float, "from": str, "to": str}
    """
    RATES = {
        "USD": {"USD": 1.0, "EUR": 0.92, "GBP": 0.78},
        "EUR": {"USD": 1.09, "EUR": 1.0, "GBP": 0.85},
        "GBP": {"USD": 1.28, "EUR": 1.18, "GBP": 1.0},
    }
    rate = RATES.get(from_code, {}).get(to_code)
    if rate is None:
        raise ValueError(f"Unsupported pair {from_code}->{to_code}")
    return {"amount": round(amount * rate, 2), "rate": rate, "from": from_code, "to": to_code}

print("✅ Tool functions defined!")

✅ Tool functions defined!


In [9]:
# OpenAI Tool Schemas

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "calculate_compound_interest",
            "description": "Calculate compound interest for financial planning and investment analysis.",
            "parameters": {
                "type": "object",
                "properties": {
                    "principal": {"type": "number", "description": "Initial investment amount"},
                    "rate": {"type": "number", "description": "Annual interest rate as percentage (e.g., 5 for 5%)"},
                    "time": {"type": "integer", "description": "Investment time period in years"},
                    "compounds_per_year": {"type": "integer", "description": "Number of times interest compounds per year", "default": 12}
                },
                "required": ["principal", "rate", "time"],
                "additionalProperties": False
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "convert_currency",
            "description": "Convert a money amount between USD/EUR/GBP using a small static table.",
            "parameters": {
                "type": "object",
                "properties": {
                    "amount": {"type": "number"},
                    "from_code": {"type": "string", "enum": ["USD", "EUR", "GBP"]},
                    "to_code": {"type": "string", "enum": ["USD", "EUR", "GBP"]},
                },
                "required": ["amount", "from_code", "to_code"],
                "additionalProperties": False,
            },
        },
    },
]

# Map tool names to Python callables
FUNCTIONS = {
    "calculate_compound_interest": calculate_compound_interest,
    "convert_currency": convert_currency,
}

print("✅ Tool schemas defined!")

✅ Tool schemas defined!


In [10]:
# Core Tool Execution Logic

def agentic_tool_call(tool_name: str, user_content: str, tool_args: dict):
    """Direct tool call via tool_choice with agentic response generation."""
    # 1) Ask the model to call ONLY the specified tool (no ambiguity)
    first = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": SYSTEM},
                  {"role": "user", "content": user_content}],
        tools=TOOLS,
        tool_choice={"type": "function", "function": {"name": tool_name}},
        temperature=0.0,
    )
    msg = first.choices[0].message
    tool_call = msg.tool_calls[0] if msg.tool_calls else None

    # 2) Execute the Python function
    if tool_call is None:
        raise RuntimeError("Model did not issue the expected tool call.")
    name = tool_call.function.name
    # Merge any model-suggested args with CLI-provided args (CLI wins)
    model_args = json.loads(tool_call.function.arguments or "{}")
    call_args = {**model_args, **tool_args}
    result = FUNCTIONS[name](**call_args)

    # 3) Send tool result back and get final assistant reply
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": msg.content or "", "tool_calls": [tool_call.model_dump()]},
        {"role": "tool", "tool_call_id": tool_call.id, "name": name, "content": json.dumps(result)},
    ]
    final = client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=0.2)
    return final.choices[0].message.content

print("✅ Core execution logic defined!")

✅ Core execution logic defined!


In [13]:
# Test Functions for Interactive Use

def test_compound_interest(principal, rate, time, compounds_per_year=12):
    """Test the compound interest calculator with given parameters."""
    prompt = f"Calculate compound interest for ${principal} at {rate}% annual rate for {time} years. Explain the result clearly. Don't include formulas or LaTeX"
    response = agentic_tool_call(
        tool_name="calculate_compound_interest",
        user_content=prompt,
        tool_args={"principal": principal, "rate": rate, "time": time, "compounds_per_year": compounds_per_year},
    )
    print(response)

def test_currency_converter(amount, from_code, to_code):
    """Test the currency converter with given parameters."""
    prompt = f"Convert {amount} {from_code} to {to_code} and explain the rate used in one sentence."
    response = agentic_tool_call(
        tool_name="convert_currency",
        user_content=prompt,
        tool_args={"amount": amount, "from_code": from_code, "to_code": to_code},
    )
    print(response)

print("✅ Test functions ready!")

✅ Test functions ready!


In [14]:
# Example Usage - Test Both Tools

print("🧪 Testing Compound Interest Calculator...")
test_compound_interest(1000, 5, 10, 12)

print("\n" + "="*60 + "\n")

print("🧪 Testing Currency Converter...")
test_currency_converter(125, "USD", "EUR")

🧪 Testing Compound Interest Calculator...
After 10 years, an investment of $1,000 at an annual interest rate of 5% will grow to approximately $1,647.01. This means that the total interest earned over this period is about $647.01.

The result shows how compound interest works: instead of just earning interest on the initial amount (the principal), you also earn interest on the interest that has been added to the principal over the years. This leads to a greater total return compared to simple interest, where only the principal earns interest. In this case, the compounding effect significantly increases the total amount over the decade.


🧪 Testing Currency Converter...
125 USD is approximately 115 EUR, using an exchange rate of 0.92.
